## MSDS 452
## Assignment 2

## Airlines flying among USA airports

In [ ]:
import random, math, numpy as np, matplotlib.pyplot as plt, pycountry, awoc, pandas as pd  
from collections import Counter
import networkx as nx
from networkx.algorithms import bipartite
from networkx.algorithms import community
import community as louvain

import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_agraph import to_agraph

import bokeh
from bokeh.io import push_notebook, show, output_notebook, save
import bokeh.plotting as bp
from bokeh.plotting import figure, save, output_file, show #, from_networkx
from bokeh.models import (ColumnDataSource, LabelSet, Label, BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,)
from holoviews.element.graphs import layout_nodes
# bokeh.sampledata.download()
from bokeh.sampledata.airport_routes import routes, airports

output_notebook()
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh', 'matplotlib')
from holoviews.operation import  gridmatrix
from holoviews.operation.datashader import datashade, bundle_graph
from holoviews import Graph, Nodes
from holoviews.plotting.bokeh import GraphPlot, LabelsPlot
import hvplot.networkx as hvnx
import hvplot.pandas

In [ ]:
title="USA Flights Graph"
routes=hv.Dataset(routes,'Airline',["Source","Destination"])
routes_df = hv.Graph(routes, ['Source', "Destination"], 'Airline', label=title).data
print(len(routes_df),list(routes_df.columns))
routes_df=routes_df[["Airline","Source","Destination"]]
routes_df

In [ ]:
acodes_df=pd.read_csv("aircodes2.csv")
acodes_d=dict(zip(acodes_df.Code, acodes_df.Airline))
# acodes_d
acodes_i_d={v:k for k,v in acodes_d.items()}
acodes_i_d

In [ ]:
airports_df=pd.read_csv("airports2.csv")
airports_d=dict(zip(airports_df.Code, airports_df.Name))
# airports_d
airports_i_d={v:k for k,v in airports_d.items()}
airports_i_d

In [ ]:
dedges=[]
for i in range(len(routes_df)):
    x=routes_df.iloc[i]["Source"]
    y=routes_df.iloc[i]["Destination"]
    z=routes_df.iloc[i]["Airline"]
    dedges.append((airports_d[x],airports_d[y],acodes_d[z]))
G=nx.DiGraph()
for k in dedges:
    G.add_edge(k[0],k[1],airline=k[2])
print("No. of nodes of the %s is %i and no. of edges is %i" %(title,len(G),len(G.edges)))

In [ ]:
def hvnx_plot(G,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
              title,bundled,nodelabels,xoffset,yoffset,
              arrowhead_length,selection_mode,selection_policy,
              edge_hover_line_color,node_hover_fill_color,
              fontsize,text_font_size, text_color,bgcolor):
    if nx.is_directed(G)==True:
        arriving_airlines={}
        for n in G.nodes():
            t=[]
            for e in G.edges(data=True):
                if n==e[1]:
                    if e[2]["airline"] not in t:
                        t.append(e[2]["airline"])
            arriving_airlines[n]=", ".join(sorted([acodes_i_d[x] for x in sorted(t)])) 
        departing_airlines={}
        for n in G.nodes():
            t=[]
            for e in G.edges(data=True):
                if n==e[0]:
                    if e[2]["airline"] not in t:
                        t.append(e[2]["airline"])
            departing_airlines[n]=", ".join(sorted([acodes_i_d[x] for x in sorted(t)])) 
        arriving_flights={}
        for n in G.nodes():
            t=list(G.predecessors(n))
            arriving_flights[n]=", ".join(sorted([airports_i_d[x] for x in t]))
        departing_flights={}
        for n in G.nodes():
            t=list(G.successors(n))
            departing_flights[n]=", ".join(sorted([airports_i_d[x] for x in t]))
        if ctype==1:
            communities_generator = community.girvan_newman(G)
            top_level_communities = next(communities_generator)
            next_level_communities = next(communities_generator)
            lc=sorted(sorted(map(sorted, next_level_communities)), key=len,reverse=True)
            partition={n:i for i,c in enumerate(lc) for n in c }
            print("No. of Girvan-Newman communities",len(set(partition.values())))
            for n in G.nodes():
                G.nodes[n]['Girvan_Newman_community'] = partition[n]
                if (n,n) in G.edges():
                    G.nodes[n]['in-degree'] = 0
                    G.nodes[n]['in-neighbors'] = ""
                    G.nodes[n]['out-degree'] = 0
                    G.nodes[n]['out-neighbors'] = ""
                else:    
                    G.nodes[n]['in-degree'] = G.in_degree(n)
                    G.nodes[n]['arriving flights from airports'] = arriving_flights[n] 
                    G.nodes[n]['arriving airlines'] = arriving_airlines[n] 
                    G.nodes[n]['out-degree'] = G.out_degree(n)
                    G.nodes[n]['departing flights to airports'] = departing_flights[n] 
                    G.nodes[n]['departing airlines'] = departing_airlines[n] 
            graph = hvnx.draw(G, pos)
            graph.opts(edge_color=edge_color,edge_line_width=edge_line_width,node_size=node_size,node_color='Girvan_Newman_community',node_cmap=node_cmap)
            if bundled==0:                
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length) #,tools=tools) #,'box_zoom',"tap"])
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
            else:
                graph = bundle_graph(graph)
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length) #,tools=tools) #,'box_zoom',"tap"])
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
        if ctype==0:
            print("Louvain communities are not computable for directed graphs!")
    else:
        connections={}
        for n in G.nodes():
            if n in acodes_d.values():
                t=list(G.neighbors(n))
                connections[n]=", ".join(sorted([airports_i_d[x] for x in t]))
            if n in airports_d.values():
                t=list(G.neighbors(n))
                connections[n]=", ".join(sorted([acodes_i_d[x] for x in t]))
        if ctype==1:
            communities_generator = community.girvan_newman(G)
            top_level_communities = next(communities_generator)
            next_level_communities = next(communities_generator)
            lc=sorted(sorted(map(sorted, next_level_communities)), key=len,reverse=True)
            partition={n:i for i,c in enumerate(lc) for n in c }
            print("No. of Girvan-Newman communities",len(set(partition.values())))
            for n in G.nodes():
                G.nodes[n]['Girvan_Newman_community'] = partition[n]
                if (n,n) in G.edges():
                    G.nodes[n]['degree'] = 0
                    G.nodes[n]['connections'] = ""
                else:    
                    G.nodes[n]['degree'] = G.degree(n)
                    G.nodes[n]['connections'] = connections[n] #", ".join([str(x) for x in list(G.neighbors(n))])
            graph = hvnx.draw(G, pos)
            graph.opts(edge_color=edge_color,edge_line_width=edge_line_width,node_size=node_size,node_color='Girvan_Newman_community',node_cmap=node_cmap)
            graph.opts(padding=0.15)
            if bundled==0:
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length) #,tools=tools) #,'box_zoom',"tap"])
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))            
                    return graph
                else:
                    return graph
            else:
                graph = bundle_graph(graph)
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length) #,tools=tools) #,'box_zoom',"tap"])
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
        if ctype==0:
            partition=louvain.best_partition(G)
            print("No. of Louvain communities",len(set(partition.values())))
            for n in G.nodes():
                G.nodes[n]['Louvain_community'] = partition[n]
                if (n,n) in G.edges():
                    G.nodes[n]['degree'] = 0
                    G.nodes[n]['connections'] = ""
                else:    
                    G.nodes[n]['degree'] = G.degree(n)
                    G.nodes[n]['connections'] = connections[n] #", ".join([str(x) for x in list(G.neighbors(n))])  #connections          
            graph = hvnx.draw(G, pos)
            graph.opts(edge_color=edge_color,edge_line_width=edge_line_width,node_size=node_size,node_color="Louvain_community",node_cmap=node_cmap)
            graph.opts(padding=0.15)
            if bundled==0:                
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length) #,tools=tools) #,'box_zoom',"tap"])
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph
            else:
                graph = bundle_graph(graph)
                graph.opts(selection_policy=selection_policy,title=title,edge_hover_line_color=edge_hover_line_color,node_hover_fill_color=node_hover_fill_color,fontsize=fontsize,width=width,height=height,arrowhead_length=arrowhead_length) #,tools=tools) #,'box_zoom',"tap"])
                if nodelabels==1:
                    labels = hv.Labels(graph.nodes, ['x', 'y'], 'index')
                    graph=(graph * labels.opts(xoffset=xoffset, yoffset=yoffset,text_font_size=text_font_size, text_color=text_color, bgcolor=bgcolor))
                    return graph
                else:
                    return graph

In [ ]:
ctype=1
node_color='Girvan_Newman_community'

bundled=1
nodelabels=0

width=1000
height=1000
pos=graphviz_layout(G)
node_size=3*np.log(5+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='lightgreen'
edge_line_width=1
title=title+" (colored in Girvan-Newman communities)"
arrowhead_length=0.01
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='4pt' #'8pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-17

hvnx_plot(G,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
ctype=1
node_color='Girvan_Newman_community'

bundled=0
nodelabels=1

width=1000
height=1000
pos=graphviz_layout(G)
node_size=3*np.log(5+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='lightgreen'
edge_line_width=1
title=title #+" (colored in Girvan-Newman communities)"
arrowhead_length=0.01
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='4pt' #'8pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-17

hvnx_plot(G,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
sorted(G.degree, key=lambda x: x[1], reverse=True)

### Question 1: 
### Select randomly a list of 50 airports and plot the corresponding subgraph of airports-airlines.

Let $G$ be a graph on a set of nodes $V$ and let $X$ a subset of $V$. Then the networkx command to generate the subgraph of $A$ is $GA = G.subgraph(A)$ 
https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.subgraph.html.
For the random selection, you may use $A = random.sample(V, 50)$, where $V = G.nodes()$. Make sure that the (sub)graph $GA$ is not trivial.

In [ ]:
A=random.sample(G.nodes(),50)
GA=G.subgraph(A)
print(len(GA),len(GA.edges()))

In [ ]:
ctype=1
node_color='Girvan_Newman_community'

bundled=0
nodelabels=1

width=700
height=700
pos=graphviz_layout(GA)
node_size=10*np.log(5+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='lightgreen'
edge_line_width=1
title="The subgraph of 50 random nodes of the "+title
arrowhead_length=0.02
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='9pt' #'8pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-17

hvnx_plot(GA,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

### Question 2: 
### Select 3 airports (either randomly or choose the ones that you have a preference for) and plot the corresponding subgraph of the egocentric networks which are centered at these airports.

If $n0$ is a node in a directed graph $G$, this is how to find the egocentric network centered at $n0$:

(1) First find in_n0 = list(G.predecessors(n0)) and out_n0 = list(G.successors(n0))

(2) Next find all neighbors_n0 = list(set(list(G.predecessors(n0)) + list(G.successors(n0))+[n0]))

(3) Finally get G.subgraph(neighbors_n0)

Do the above for each one of the 3 egos and finally get G.subgraph(list(set(neighbors_n0+neighbors_n1+neghbors_n2)))

In [ ]:
airportslist=['El Paso International Airport','Fairbanks International Airport','Fort Wayne International Airport'] # ADD 3 AIRPORTS IN THIS LIST BUT HAVING LOW NUMBERS OF FLIGHTS!
# YOU NEED TO CHANGE THIS LIST ADDING 3 AIRPORTS IN IT HAVING LOW NUMBERS OF FLIGHTS!
alle=[]
for a in airportslist:
    for b in set(list(G.predecessors(a)) + list(G.successors(a))+[a]):
        alle.append(b)
GAA=G.subgraph(alle)
print(len(GAA),len(GAA.edges()))

In [ ]:
ctype=1
node_color='Girvan_Newman_community'

bundled=1
nodelabels=0

width=700
height=700
pos=graphviz_layout(GAA)
node_size=4*np.log(5+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='lightgreen'
edge_line_width=1
title="The subgraph of 3 egocentic networks"
arrowhead_length=0.02
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='9pt' #'8pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-17

hvnx_plot(GAA,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
bedges=[]
for e in G.edges(data=True):
    if (e[0],e[2]['airline']) not in bedges:
        bedges.append((e[0],e[2]['airline']))
    if (e[1],e[2]['airline']) not in bedges:
        bedges.append((e[1],e[2]['airline']))
# print(len(bedges),len(set(bedges)))
H=nx.Graph()
H.add_edges_from(bedges)
title1="Bipartite Graph of USA Airlines vs. Airports"
airlines=[n for n in H.nodes() if n in acodes_d.values()]
airports=[n for n in H.nodes() if n in airports_d.values()]
print("No. of nodes of the %s is %i (%i airlines and %i airports) and no. of edges is %i" %(title1,len(H),len(airlines),len(airports),len(H.edges)))

In [ ]:
Ha=H.subgraph(airlines)
Hb=H.subgraph(airports)
posa=nx.circular_layout(Ha)
posb=nx.spring_layout(Hb,0.003)
def re_pos(pos,centre,radius):
    plist=[list(v) for v in pos.values()]
    nplist=[v for v in pos.values()]
    x = [p[0] for p in plist]
    y = [p[1] for p in plist]
    centroid = (sum(x) / float(len(plist)), sum(y) / float(len(plist)))
    ncentroid=np.array(centroid)
    rad=max([np.linalg.norm(nplist[i]-ncentroid) for i in range(len(nplist))])
    rpos={}
    for k,v in pos.items():
        rpos[k]=[(v[0]+centre[0])/float(radius),(v[1]+centre[1])/float(radius)]
    return rpos
posa=re_pos(posa,[0,0],1)
posb=re_pos(posb,[0,0],1.5)
posBipartite=dict(list(posb.items())+list(posa.items())+[(k,posb[k]+posa[k]) for k in set(posa)&set(posb)])

In [ ]:
ctype=1
node_color='Girvan-Newman_community'

bundled=0
nodelabels=1

width=1000
height=1000
pos=posBipartite
node_size=5*np.log(5+hv.dim('degree')) #4*np.log(6+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='olive'
edge_line_width=1
title=title1+" (colored in Girvan-Newman communities)"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='4pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-0.03 #-15
    
hvnx_plot(H,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
ctype=1
node_color='Girvan-Newman_community'

bundled=1
nodelabels=0

width=1000
height=1000
pos=posBipartite
node_size=5*np.log(5+hv.dim('degree')) #4*np.log(6+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='olive'
edge_line_width=1
title=title1 #+" (colored in Girvan-Newman communities)"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='4pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-0.03 #-15
    
hvnx_plot(H,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
ctype=0
node_color='Louvain_community'

bundled=0
nodelabels=1

width=1000
height=1000
pos=posBipartite
node_size=5*np.log(5+hv.dim('degree')) #4*np.log(6+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='olive'
edge_line_width=1
title=title1+" (colored in Louvain communities)"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='4pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-0.03 #-15
    
hvnx_plot(H,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
ctype=0
node_color='Louvain_community'

bundled=1
nodelabels=0

width=1000
height=1000
pos=posBipartite
node_size=5*np.log(5+hv.dim('degree')) #4*np.log(6+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='olive'
edge_line_width=1
title=title1 #+" (colored in Louvain communities)"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='4pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-0.03 #-15
    
hvnx_plot(H,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)

In [ ]:
for i in sorted(H.degree, key=lambda x: x[1], reverse=True):
    if i[0] in airlines:
        print(i)

### Question 3: 
### Select a list of 10 airlines (either randomly or choose the ones that you have a preference for)  together with the airports to which these airlines fly. Take the corresponding  subgraph in the bipartite graph of airlines vs. airports and plot it.

In [ ]:
AA=A=['Bering Air',
 'Great Lakes Airlines',
 'Virgin Atlantic Airways',
 'Spirit Airlines',
 'Frontier Airlines', 
 'SeaPort Airlines',
 'Aero Flight',
 'Virgin America',
 'Peninsula Airways',
 'Wright Air Service']
# YOU NEED TO CHANGE THIS LIST ENTERING 10 OTHER AIRLINES IN IT
# BUT CONSIDER ONLY THOSE AIRLINES HAVING LOW NUMBERS OF FLIGHTS!
# CHECK OUT AIRLINES IN THE PREVIOUS CELL.
for a in AA:
    if b in H.neighbors(a):
        AA.append(b)
HA=H.subgraph(AA)
print(len(HA),len(HA.edges()))

In [ ]:
ctype=0
node_color='Girvan-Newman_community'

bundled=0
nodelabels=1

width=700
height=700
pos=graphviz_layout(HA) #posBipartite
node_size=5*np.log(5+hv.dim('degree')) #4*np.log(6+hv.dim('in-degree'))
node_cmap="tab10"
edge_color='olive'
edge_line_width=1
title="Connecting flights between a random set of 10 airports"
arrowhead_length=0.04
selection_mode='nodes'
selection_policy="nodes"
edge_hover_line_color='green'
node_hover_fill_color='red'
fontsize={'title': '9pt'}
text_font_size='10pt'
text_color='black'
bgcolor='white'
xoffset=0
yoffset=-10 #-15
    
hvnx_plot(HA,ctype,pos,width,height,node_size,node_cmap,edge_color,edge_line_width,
             title,bundled,nodelabels,xoffset,yoffset,
             arrowhead_length,selection_mode,selection_policy,
             edge_hover_line_color,node_hover_fill_color,
             fontsize,text_font_size, text_color,bgcolor)